In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
import gc
import os

import torch


def clear():
    """Clear GPU memory"""
    visible_devices = os.getenv("CUDA_VISIBLE_DEVICES")
    print(f"Clearing GPU memory for {visible_devices}")
    torch.cuda.empty_cache()
    gc.collect()

In [4]:
from datasets import load_dataset

# ds = load_dataset('openai/gsm8k', 'main', split='train[:100]').map(
#     lambda x: {"prompt": x["question"], "answer": x["answer"]}
# )
ds = load_dataset("Idavidrein/gpqa", "gpqa_diamond", split="train")
ds = ds.map(
    lambda x: {"prompt": x["Question"], "answer": x["Correct Answer"]}, remove_columns=ds.column_names
)
ds[0]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

{'prompt': 'Two quantum states with energies E1 and E2 have a lifetime of 10^-9 sec and 10^-8 sec, respectively. We want to clearly distinguish these two energy levels. Which one of the following options could be their energy difference so that they can be clearly resolved?\n',
 'answer': '10^-4 eV'}

In [5]:
from vllm import LLM
from vllm.sampling_params import SamplingParams

from verifiers.utils.cuda import get_half_precision_dtype


# clear()

# model = 'Qwen/QwQ-32B'
# model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
# model = "meta-llama/Llama-3.1-8B-Instruct"
model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

llm = LLM(
    model=model,
    dtype=get_half_precision_dtype(),
    gpu_memory_utilization=0.8,
    max_model_len=4096,
    trust_remote_code=True,
)


INFO 03-21 22:14:42 [__init__.py:256] Automatically detected platform cuda.
[2025-03-21 22:14:44,494] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.3/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: /usr/local/cuda-12.3/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/baris/miniconda3/envs/verifiers/compiler_compat/ld: /usr/local/cuda-12.3/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_

INFO 03-21 22:14:52 [config.py:583] This model supports multiple tasks: {'classify', 'generate', 'reward', 'embed', 'score'}. Defaulting to 'generate'.
INFO 03-21 22:14:52 [config.py:1693] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 03-21 22:14:54 [utils.py:2148] CUDA was previously initialized. We must use the `spawn` multiprocessing start method. Setting VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information.
INFO 03-21 22:14:57 [__init__.py:256] Automatically detected platform cuda.
INFO 03-21 22:14:59 [core.py:53] Initializing a V1 LLM engine (v0.8.1) with config: model='deepseek-ai/DeepSeek-R1-Distill-Llama-8B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Llama-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_se

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.28s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.40s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.39s/it]



INFO 03-21 22:15:04 [loader.py:429] Loading weights took 3.13 seconds
INFO 03-21 22:15:04 [gpu_model_runner.py:1176] Model loading took 14.9888 GB and 4.121359 seconds
INFO 03-21 22:15:11 [backends.py:409] Using cache directory: /home/baris/.cache/vllm/torch_compile_cache/48a4bdad36/rank_0_0 for vLLM's torch.compile
INFO 03-21 22:15:11 [backends.py:419] Dynamo bytecode transform time: 7.20 s
INFO 03-21 22:15:12 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 03-21 22:15:18 [monitor.py:33] torch.compile takes 7.20 s in total
INFO 03-21 22:15:19 [kv_cache_utils.py:537] GPU KV cache size: 352,864 tokens
INFO 03-21 22:15:19 [kv_cache_utils.py:540] Maximum concurrency for 4,096 tokens per request: 86.15x
INFO 03-21 22:15:39 [gpu_model_runner.py:1499] Graph capturing finished in 20 secs, took 1.61 GiB
INFO 03-21 22:15:39 [core.py:138] init engine (profile, create kv cache, warmup model) took 35.24 seconds


In [6]:
tokenizer = llm.get_tokenizer()
tokenizer.chat_template = """{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% set ns = namespace(is_first=false, is_tool=false, is_output_first=true, system_prompt='') %}{%- for message in messages %}{%- if message['role'] == 'system' %}{% set ns.system_prompt = message['content'] %}{%- endif %}{%- endfor %}{{bos_token}}{{ns.system_prompt}}{%- for message in messages %}{%- if message['role'] == 'user' %}{%- set ns.is_tool = false -%}{{'<｜User｜>' + message['content']}}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is none %}{%- set ns.is_tool = false -%}{%- for tool in message['tool_calls']%}{%- if not ns.is_first %}{{'<｜Assistant｜><｜tool▁calls▁begin｜><｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\\n' + '```json' + '\\n' + tool['function']['arguments'] + '\\n' + '```' + '<｜tool▁call▁end｜>'}}{%- set ns.is_first = true -%}{%- else %}{{'\\n' + '<｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\\n' + '```json' + '\\n' + tool['function']['arguments'] + '\\n' + '```' + '<｜tool▁call▁end｜>'}}{{'<｜tool▁calls▁end｜><｜end▁of▁sentence｜>'}}{%- endif %}{%- endfor %}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is not none %}{%- if ns.is_tool %}{{'<｜tool▁outputs▁end｜>' + message['content'] + '<｜end▁of▁sentence｜>'}}{%- set ns.is_tool = false -%}{%- else %}{{'<｜Assistant｜>' + message['content'] + '<｜end▁of▁sentence｜>'}}{%- endif %}{%- endif %}{%- if message['role'] == 'tool' %}{%- set ns.is_tool = true -%}{%- if ns.is_output_first %}{{'<｜tool▁outputs▁begin｜><｜tool▁output▁begin｜>' + message['content'] + '<｜tool▁output▁end｜>'}}{%- set ns.is_output_first = false %}{%- else %}{{'\\n<｜tool▁output▁begin｜>' + message['content'] + '<｜tool▁output▁end｜>'}}{%- endif %}{%- endif %}{%- endfor -%}{% if ns.is_tool %}{{'<｜tool▁outputs▁end｜>'}}{% endif %}{% if add_generation_prompt and not ns.is_tool %}{{'<｜Assistant｜><think>\\n'}}{% endif %}"""
llm.set_tokenizer(tokenizer)

In [28]:
example = ds[9]
prompt = example['prompt']
print(prompt)
print()
print("Answer: ", example['answer'])

Among the following exoplanets, which one has the highest density?

a) An Earth-mass and Earth-radius planet.
b) A planet with 2 Earth masses and a density of approximately 5.5 g/cm^3.
c) A planet with the same composition as Earth but 5 times more massive than Earth.
d) A planet with the same composition as Earth but half the mass of Earth.

Answer:  c


In [29]:
def chat_completion(messages, max_tokens=1024, temperature=0.5, top_p=0.95, **kwargs):
    """
    Perform chat completion with the given messages and token limit.

    Args:
        messages: List of message dictionaries with role and content
        max_tokens: Maximum tokens to generate in response

    Returns:
        Generated text from the model
    """
    sampling_params = SamplingParams(
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        **kwargs,
    )
    continue_final_message = messages[-1]['role'] == 'assistant'

    outputs = llm.chat(
        messages,
        sampling_params,
        continue_final_message=continue_final_message,
        add_generation_prompt=not continue_final_message,
        use_tqdm=False,
    )
    return outputs[0].outputs[0].text

In [31]:
completion_prefix = "<think>...</think>"

messages = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": completion_prefix},
]
print(chat_completion(messages, max_tokens=1024))


Alright, so I have this question about the density of exoplanets and I need to figure out which one is the highest. Let me break it down step by step.

First, let's understand the question. It gives four options, a) through d), each describing a different planet with different masses and densities. I need to determine which has the highest density. Density is mass divided by volume, right? So, density = mass / volume. But since we're dealing with exoplanets, I might not have all the exact data, so I'll need to make some assumptions or use some formulas.

Let me list out the options again:

a) An Earth-mass and Earth-radius planet.

b) A planet with 2 Earth masses and a density of approximately 5.5 g/cm³.

c) A planet with the same composition as Earth but 5 times more massive than Earth.

d) A planet with the same composition as Earth but half the mass of Earth.

Hmm, okay. So, options a and d mention Earth's mass and radius or half of it. Options b gives a density, and option c talks

In [32]:
# messages = [
#     {"role": "user", "content": "Hello"},
#     {"role": "assistant", "content": "<think>...</think>"},
# ]
# tokenizer.apply_chat_template(messages, tokenize=False, continue_final_message=False)
# tokenizer.apply_chat_template(messages, tokenize=False, continue_final_message=True)

In [33]:
# compare baseline completion vs thinking budget completion by completion length and answer correctness
# Function to get baseline completion without thinking budget
def generate_baseline(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]
    return chat_completion(messages)

In [34]:
response = generate_baseline(prompt)
print(response)

Alright, so I have this question about exoplanets and their densities. Hmm, okay, let me try to figure this out step by step. I'm a bit rusty on some of these concepts, but I'll give it a shot.

The question is asking which of the four options has the highest density. The options are:

a) An Earth-mass and Earth-radius planet.
b) A planet with 2 Earth masses and a density of approximately 5.5 g/cm³.
c) A planet with the same composition as Earth but 5 times more massive than Earth.
d) A planet with the same composition as Earth but half the mass of Earth.

First, I need to remember what density means. Density is mass divided by volume, right? So, density = mass / volume. So, if I can figure out the volume for each planet, I can compare their densities.

But wait, how do I find the volume? Well, I know that volume is mass divided by density, so volume = mass / density. But without knowing the density, that might not help directly. Hmm, maybe I can relate the mass and radius somehow.

Oh

In [35]:
def generate_with_thinking_budget(prompt, num_steps=10, tokens_per_step=50, strict=False):
    """
    Generate a response with controlled thinking budget.

    Args:
        prompt: The user prompt to respond to
        num_steps: Number of thinking steps to use
        tokens_per_step: Maximum tokens to generate per thinking step

    Returns:
        The final generated text with thinking steps and answer
    """
    # Start with the basic conversation
    messages = [
        {"role": "user", "content": prompt},
    ]

    # Initialize with the first thinking step
    initial_content = f"<think progress=1/{num_steps}>\n"
    assistant_msg = {"role": "assistant", "content": initial_content}
    messages.append(assistant_msg)

    # Track the complete response
    full_response = initial_content

    # Generate each thinking step iteratively
    for step in range(1, num_steps + 1):
        # Generate the next chunk of thinking
        step_response = chat_completion(messages, max_tokens=tokens_per_step)

        # Update our full response with this step's generation
        full_response += step_response

        # Check if thinking has naturally concluded
        if "</think>" in step_response:
            if strict:
                # If we're in strict mode, we need to force the model to continue thinking
                step_response = step_response.replace("</think>", "\nWait, ")
            else:
                # If we're not in strict mode, we can stop here
                break

        # Prepare for the next step if we're not at the end
        if step < num_steps:
            # Update the step number in the response content
            current_tag = f"<think progress={step}/{num_steps}>"
            next_tag = f"<think progress={step + 1}/{num_steps}>"
            full_response = full_response.replace(current_tag, next_tag)

            # Update the assistant's message with the new content
            messages[-1]["content"] = full_response

    # Check if thinking completed naturally
    if "</think>" not in full_response:
        # Force close the thinking tag if it wasn't closed
        full_response += "\n</think>"
        messages[-1]["content"] = full_response

    # Remove the progress info    
    full_response = full_response.replace(f'<think progress={num_steps}/{num_steps}>', '<think>')
    messages[-1]["content"] = full_response

    # Generate the final answer with larger token limit
    final_answer = chat_completion(messages, max_tokens=f)
    full_response += final_answer

    return full_response

In [49]:
def generate_with_thinking_budget(prompt, num_steps=10, tokens_per_step=50, strict=False):
    """
    Generate a response with controlled thinking budget.

    Args:
        prompt: The user prompt to respond to
        num_steps: Number of thinking steps to use
        tokens_per_step: Maximum tokens to generate per thinking step

    Returns:
        The final generated text with thinking steps and answer
    """
    # Start with the basic conversation
    messages = [
        {"role": "user", "content": prompt},
    ]

    # Initialize with the first thinking step
    budget = num_steps * tokens_per_step
    initial_content = f"<think budget={budget}>\n"
    assistant_msg = {"role": "assistant", "content": initial_content}
    messages.append(assistant_msg)

    # Track the complete response
    full_response = initial_content

    # Generate each thinking step iteratively
    for step in range(1, num_steps + 1):
        # Generate the next chunk of thinking
        step_response = chat_completion(messages, max_tokens=tokens_per_step)

        # Update our full response with this step's generation
        full_response += step_response

        # Check if thinking has naturally concluded
        if "</think>" in step_response:
            if strict:
                # If we're in strict mode, we need to force the model to continue thinking
                step_response = step_response.replace("</think>", "\nWait, ")
            else:
                # If we're not in strict mode, we can stop here
                break

        new_budget = budget - tokens_per_step

        # Update the step number in the response content
        current_tag = f"<think budget={budget}>"
        next_tag = f"<think budget={new_budget}>"
        full_response = full_response.replace(current_tag, next_tag)
        messages[-1]["content"] = full_response        

        budget = new_budget

    # Check if thinking completed naturally
    if "</think>" not in full_response:
        # Force close the thinking tag if it wasn't closed
        full_response += "\n</think>"
        messages[-1]["content"] = full_response

    # Remove the progress info    
    # full_response = full_response.replace(f'<think budget=0>', '<think>')
    # messages[-1]["content"] = full_response

    # Generate the final answer with larger token limit
    max_tokens = (num_steps * tokens_per_step)//4
    final_answer = chat_completion(messages, max_tokens=max_tokens)
    full_response += final_answer
    return full_response

In [64]:
def generate_with_thinking_budget(prompt, num_steps=10, tokens_per_step=50, strict=False):
    """
    Generate a response with controlled thinking budget.

    Args:
        prompt: The user prompt to respond to
        num_steps: Number of thinking steps to use
        tokens_per_step: Maximum tokens to generate per thinking step

    Returns:
        The final generated text with thinking steps and answer
    """
    budget = num_steps * tokens_per_step
    
    # Start with the basic conversation
    messages = [
        {"role": "user", "content": prompt},
        {
            "role": "assistant", 
            "content": f"<think>\nI have {budget} tokens to think out a solution.",
        },
    ]

    # Generate each thinking step iteratively
    for step in range(1, num_steps + 1):
        # Generate the next chunk of thinking
        step_response = chat_completion(messages, max_tokens=tokens_per_step)
        
        # Check if thinking has naturally concluded
        if "</think>" in step_response:
            if strict:
                # If we're in strict mode, we need to force the model to continue thinking
                step_response = step_response.replace("</think>", "\nWait, ")
            else:
                # If we're not in strict mode, we can stop here
                break

        # Update the message
        new_budget = budget - tokens_per_step
        messages[-1]["content"] += step_response + f"\nBudget left: {new_budget}"
        budget = new_budget

    # Check if thinking completed naturally
    if "</think>" not in messages[-1]["content"]:
        # Force close the thinking tag if it wasn't closed
        messages[-1]["content"] += "\n</think>"

    messages[-1]["content"] = messages[-1]["content"].rstrip() + "\nFinal Answer: "

    # Generate the final answer with larger token limit
    max_tokens = (num_steps * tokens_per_step)//4
    final_answer = chat_completion(messages, max_tokens=max_tokens)
    return messages[-1]['content']

In [65]:
response = generate_with_thinking_budget(prompt, num_steps=10, tokens_per_step=100, strict=True)
print(prompt)
print('-'*80)
print(response)

Among the following exoplanets, which one has the highest density?

a) An Earth-mass and Earth-radius planet.
b) A planet with 2 Earth masses and a density of approximately 5.5 g/cm^3.
c) A planet with the same composition as Earth but 5 times more massive than Earth.
d) A planet with the same composition as Earth but half the mass of Earth.
--------------------------------------------------------------------------------
<think>
I have 1000 tokens to think out a solution. I'll start by understanding the question.

I need to determine which of the four exoplanets has the highest density. The options are:

a) Earth-mass and Earth-radius planet.
b) 2 Earth masses with a density of 5.5 g/cm³.
c) Same composition as Earth but 5 times the mass.
d) Same composition as Earth but half the mass.

First, I remember that density is mass divided by volume. So, density = mass / volume.


Budget left: 900 tokens.

I should consider the relationship between mass and volume for each option. For a given

In [55]:
ds[9]

{'prompt': 'Among the following exoplanets, which one has the highest density?\n\na) An Earth-mass and Earth-radius planet.\nb) A planet with 2 Earth masses and a density of approximately 5.5 g/cm^3.\nc) A planet with the same composition as Earth but 5 times more massive than Earth.\nd) A planet with the same composition as Earth but half the mass of Earth.',
 'answer': 'c'}